<a href="https://colab.research.google.com/github/hsshih/Kaggle_Galaxy_Classification/blob/main/Visualize_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing libraries
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.models import model_from_json
from matplotlib import pyplot
from numpy import expand_dims


In [ ]:
img_width, img_height = 212, 212

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")

for i in range(len(model.layers)):
    layer = model.layers[i]
    # check for convolutional layer
    if 'conv' not in layer.name:
        continue
    # summarize output shape
    print(i, layer.name, layer.output.shape)

# retrieve weights from the first hidden layer
filters, biases = model.layers[0].get_weights()

# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

# plot first few filters
n_filters, ix = 6, 1
for i in range(n_filters):
    # get the filter
    f = filters[:, :, :, i]
    # plot each channel separately
    for j in range(3):
        # specify subplot and turn of axis
        ax = pyplot.subplot(n_filters, 3, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        pyplot.imshow(f[:, :, j], cmap='gray')
        ix += 1
# show the figure
pyplot.show()

# redefine model to output right after the first pooling layer
model_1 = Model(inputs=model.inputs, outputs=model.layers[2].output)
model_1.summary()

# load the example image, make a prediction, and get feature maps
#img = load_img('spiral_example1.jpg', target_size=(img_width, img_height))
img = load_img('elliptical_example1.jpg', target_size=(img_width, img_height))
img = img_to_array(img)
img = expand_dims(img, axis=0)
feature_maps = model_1.predict(img)
#numpy.shape(feature_maps)

# plot all 32 maps in an 8x4 squares
rows = 4
columns = 8
ix = 1
for _ in range(rows):
    for _ in range(columns):
        # specify subplot and turn of axis
        ax = pyplot.subplot(rows, columns, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        pyplot.imshow(feature_maps[ 0, :, :, ix-1], cmap='gray')
        ix += 1
# save the figure
pyplot.savefig('features1.png')